In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

def featsrc(rp1, rn1):    
    
    res = pd.concat([rp1,rn1], axis = 0).reset_index(drop = True)
    
    rpd = pd.concat([rp1.quantile(q=0.05), rp1.quantile(q=0.10), rp1.quantile(q=0.90), rp1.quantile(q=0.95)], axis=1)
    rnd = pd.concat([rn1.quantile(q=0.05), rn1.quantile(q=0.10), rn1.quantile(q=0.90), rn1.quantile(q=0.95)], axis=1)
    
    odf = pd.DataFrame(index = rpd.index, columns= rpd.columns)

    for i3 in rpd.index:
        odf.loc[i3,0.05] = max(rpd.loc[i3,0.05], rnd.loc[i3,0.05])
        odf.loc[i3,0.10] = max(rpd.loc[i3,0.10], rnd.loc[i3,0.10])
        odf.loc[i3,0.90] = min(rpd.loc[i3,0.90], rnd.loc[i3,0.90])
        odf.loc[i3,0.95] = min(rpd.loc[i3,0.95], rnd.loc[i3,0.95])
    
    odf['in 90'] = 0
    odf['in 80'] = 0

    for i in res.index:
        for c in res.columns:
            if odf.loc[c,0.05] < res.loc[i,c] < odf.loc[c,0.95]:
                odf.loc[c, 'in 90'] = odf.loc[c, 'in 90'] + 1

                if odf.loc[c,0.10] < res.loc[i,c] < odf.loc[c,0.90]:
                    odf.loc[c, 'in 80'] = odf.loc[c, 'in 80'] + 1
    
    return odf.sort_values(by=['in 80', 'in 90'])[:200].index


#sample df1 into 440 entries
pndf = pd.read_csv('pndf.csv')
res = pd.read_csv('res 340.csv')

fdf = pd.DataFrame(index = res.columns, columns = ['challenger', 'scontrol', 'pcontrol'])
fdf['challenger'] = 0
fdf['scontrol'] = 0
fdf['pcontrol'] = 0

#challenger
for i in range(100):
    df1 = pd.concat([pndf, res], axis = 1)
    p = df1.loc[df1['Gen'] == 'ssRNA(+)'].sample(220).reset_index(drop = True)
    n = df1.loc[df1['Gen'] == 'ssRNA(-)'].sample(220).reset_index(drop = True)
    rp1 = p.iloc[:,8:]
    rn1 = n.iloc[:,8:]
    feats = featsrc(rp1,rn1)
    c = 200
    for f in feats:
        fdf.loc[f,'challenger'] = fdf.loc[f,'challenger'] + c
        c-=1
    
    #scontrol score
    df1 = pd.concat([p,n], axis = 0).reset_index(drop = True)    
    df1['Gen'] = 0
    rindex = np.random.choice(len(df1), 220, replace=False)
    for i2 in rindex:
        df1.loc[i2,'Gen'] = 1      

    p = df1.loc[df1['Gen'] == 0].sample(220).reset_index(drop = True)
    n = df1.loc[df1['Gen'] == 1].sample(220).reset_index(drop = True)
    rp1 = p.iloc[:,8:]
    rn1 = n.iloc[:,8:]
    feats = featsrc(rp1,rn1)
    c = 200
    for f in feats:
        fdf.loc[f,'scontrol'] = fdf.loc[f,'scontrol'] + c
        c-=1
    print('%d challenger iterations remaining.' % (99-i))
    
#control
for i in range(100):    
    df1 = pd.concat([pndf, res], axis = 1)
    p = df1.loc[df1['Gen'] == 'ssRNA(+)'].sample(220).reset_index(drop = True)
    n = df1.loc[df1['Gen'] == 'ssRNA(-)'].sample(220).reset_index(drop = True)
    df1 = pd.concat([p,n], axis = 0).reset_index(drop = True)
    df1['Gen'] = 0    
    rindex = np.random.choice(len(df1), 220, replace=False)
    for i2 in rindex:
        df1.loc[i2,'Gen'] = 1   
    p = df1.loc[df1['Gen'] == 0].sample(220).reset_index(drop = True)
    n = df1.loc[df1['Gen'] == 1].sample(220).reset_index(drop = True)
    rp1 = p.iloc[:,8:]
    rn1 = n.iloc[:,8:]

    feats = featsrc(rp1,rn1)
    c = 200
    for f in feats:
        fdf.loc[f,'pcontrol'] = fdf.loc[f,'pcontrol'] + c
        c-=1
    print('%d control iterations remaining.' % (99 - i))

99 challenger iterations remaining.
98 challenger iterations remaining.
97 challenger iterations remaining.
96 challenger iterations remaining.
95 challenger iterations remaining.
94 challenger iterations remaining.
93 challenger iterations remaining.
92 challenger iterations remaining.
91 challenger iterations remaining.
90 challenger iterations remaining.
89 challenger iterations remaining.
88 challenger iterations remaining.


In [45]:
srtf = fdf.sort_values(by=['result', 'challenger'], ascending = [False, False])

In [57]:
srtf.loc[srtf['challenger'] > 9000][:10].index

Index(['GGCA', 'GCCA', 'ACG', 'TGAT', 'TCTA', 'TTAA', 'ATTC', 'G', 'TAAG',
       'TTAT'],
      dtype='object')

In [ ]:
def split(df1):
    p = df1.loc[:219].sample(frac=1).reset_index(drop = True)
    n = df1.loc[220:].sample(frac=1).reset_index(drop = True)
    ps1 = p.iloc[:88]
    ns1 = n.iloc[:88]
    ps2 = p.iloc[88:176].reset_index(drop = True)
    ns2 = n.iloc[88:176].reset_index(drop = True)
    ps3 = p.iloc[176:].reset_index(drop = True)
    ns3 = n.iloc[176:].reset_index(drop = True)
    
    return (ps1,ns1,ps2,ns2,ps3,ns3)